# FDG Late-Breaking Work Experiment Section
Uses the two AI generations and human-made (extracted from Steam descriptions) text as comparison for real and fake games to ask users which set they prefer

#### IMPORTS AND SETUP

In [21]:
import os
import torch
import random
from tqdm import tqdm

# other files
import sys
sys.path.append('../')
import Python.utils as utils
import Python.GAME_FEATURE_REC_MOD as gfr_mod

In [22]:
# setup the game recommender
GRMOD = gfr_mod.GameRecMod()
GRMOD.setup()

Importing full game data...


Importing GloVe: 100%|██████████| 400000/400000 [00:05<00:00, 66821.02it/s]


In [32]:
# import the transformer model
from transformers import GPT2LMHeadModel, GPT2Tokenizer

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import_model_path = "../models/transformers/feature_generator"
model_name = "gpt2"

feat_tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',eos_token='<|endoftext|>', pad_token='<|pad|>')
feat_gen_model = GPT2LMHeadModel.from_pretrained(import_model_path).to(DEVICE)
feat_gen_model.resize_token_embeddings(len(feat_tokenizer))
_ = feat_gen_model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### PREDICTION CODE

In [98]:
def getEntTags(prompt,shuffle=False):
    #remove all stop words
    tokens = utils.tokenize(prompt)

    #separate for the tags and entities
    tags = [t for t in tokens if t in GRMOD.ALL_TAGS]
    entities = [e for e in tokens if not e in tags]

    if shuffle:
        random.shuffle(tags)
        random.shuffle(entities)
    return tags, entities

In [112]:
# for the transformer generator

# predict features from a transformer model given a prompt
def predictTrans(prompt, args={}):
    tags, entities = getEntTags(prompt,shuffle=True)
    tag_str = ",".join(tags)
    entity_str = ",".join(entities)
    input_txt = f"<|startoftext|>{tag_str}\n{entity_str}\n-"
    generated = feat_tokenizer(input_txt, return_tensors="pt").input_ids.to(DEVICE)

    # perform prediction
    MAX_LEN = 512
    top_k = args['top_k'] if 'top_k' in args else 50
    top_p = args['top_p'] if 'top_p' in args else 1.0
    temperature = args['temperature'] if 'temperature' in args else 0.9
    repetition_penalty = args['rep_pen'] if 'rep_pen' in args else 0.95
    
    print(f"k {top_k}, p: {top_p}, temp: {temperature}")
    
    sample_outputs = feat_gen_model.generate(generated, do_sample=True, top_k=top_k, max_length=512, top_p=top_p, temperature=temperature, repetition_penalty = repetition_penalty, pad_token_id=feat_tokenizer.convert_tokens_to_ids('<|pad|>'))

    # decode the predicted tokens into texts
    pred_text = feat_tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

    # print/return the predicted text
    # print(pred_text)
    return pred_text

# predict features from a transformer model given a set of tokens
def predictTrans2(tags, entities, args={}):
    tag_str = ",".join(tags)
    entity_str = ",".join(entities)
    input_txt = f"<|startoftext|>{tag_str}\n{entity_str}\n-"
    generated = feat_tokenizer(input_txt, return_tensors="pt").input_ids.to(DEVICE)

    # perform prediction
    MAX_LEN = 512
    top_k = args['top_k'] if 'top_k' in args else 50
    top_p = args['top_p'] if 'top_p' in args else 1.0
    temperature = args['temperature'] if 'temperature' in args else 0.9
    repetition_penalty = args['rep_pen'] if 'rep_pen' in args else 0.95
    # repetition_penalty = args['rep_pen'] if 'rep_pen' in args else 0.05

    
    print(f"k {top_k}, p: {top_p}, temp: {temperature}")
    
    sample_outputs = feat_gen_model.generate(generated, do_sample=True, top_k=top_k, max_length=512, top_p=top_p, temperature=temperature, repetition_penalty = repetition_penalty, pad_token_id=feat_tokenizer.convert_tokens_to_ids('<|pad|>'))

    # decode the predicted tokens into texts
    pred_text = feat_tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

    # print/return the predicted text
    # print(pred_text)
    return pred_text


# for the concept net generator
def predictConNet(prompt):
    tags, entities = getEntTags(prompt)
    tokens = tags + entities

    # set up the feature recommender
    FMOD = gfr_mod.FeatureMod(tokens,get_related=False)
    return [f"- {f}" for f in FMOD.generate(15)], FMOD

def predictConNet2(tokens):
    # set up the feature recommender
    FMOD = gfr_mod.FeatureMod(tokens,get_related=False)
    return [f"- {f}" for f in FMOD.generate(15)], FMOD


#### PROMPTS AND GAME SELECTION

In [113]:
# get the human authored game features from real games pulled from Steam
real_games = [
    {
        "name": "TEAM FORTRESS 2",
        "prompt": "a class based multiplayer online shooter with a variety of modes including capture the flag, death match, and deliver the payload",
        "tags": [],
        "entities": [],
        "human_features": ["- shoot gun to heal allies", "- double jump in the air", "- drink energy drinks", "- throw suspicious jars", "- backstab enemy targets", "- sneak across terrain"]
    },
    {
        "name": "DOTA 2",
        "prompt": "a 5v5 game where you protect your base while destroying enemy bases using an array of different abilities and items",
        "tags": [],
        "entities": [],
        "human_features": ["- kill npc enemies", "- buy power ups", "- type to communicate with enemy team", "- coordinate tactics with allies", "- protect allied mobs", "- attack enemy turrets"]
    },
    {
        "name": 'THE LEGEND OF ZELDA: BREATH OF THE WILD',
        "prompt": "an open-world exploration game in a post-apocalyptic fantasy world where you can climb anything and destroy everything",
        "tags": [],
        "entities": [],
        "human_features": ["- explore the wilds", "- battle towering enemies", "- think quickly and develop the right strategies", "- gather ingredients", "- earn special items", "- bundle up with warmer clothes"]
    }
]

# get the human authored game features from fake games
fake_games = [
    {
        "name": "ALCHEMY PRINCESS",
        "prompt": "an RPG about a princess who collects swords and flowers to turn into potions and is secretly a frog",
        "tags": [],
        "entities": [],
        "human_features": ["- fight sludge monsters in the swamp", "- discover new alchemy recipes", "- identify new flora and fauna", "- explore the ruined castle", "meet new animal friends", "find the antidote to your curse"]
    },
    {
        "name": "THE ONIGIRI DORM",
        "prompt": "a collaborative cooking game where you make and sell onigiri in your college dorm room",
        "tags": [],
        "entities": [],
        "human_features": ["- make onigiri on the weekends", "- set your own prices", "- buy new meats and veggies", "- decorate your dorm room", "- pay off your tuition", "- hire friends and roommates part-time"]
    },
    {
        "name": "HOVERDECK 209X",
        "prompt": "a retro-futuristic cyberpunk skateboarding game where you hack corporations, the robot police, and the street gangs",
        "tags": [],
        "entities": [],
        "human_features": ["- hack the corpo suits", "- vandalize properties with graffiti", "- find secret hideouts", "- race the street gangs", "- upgrade your bioware tech", "- skate the streets of the city"]
    }
]


In [114]:
for game in real_games:
    t = GRMOD.getGameTags(game['name'])
    e = GRMOD.getGameEntities(game['name'])
    game['tags'] = t
    game['entities'] = e

# set the entities and tags for each fake game
# fake_game_rec_set = [['ARCUS SPIRITS'], ['QUIZ TORIMONOCHOU'], ['WAVE BREAK']]
# fake_game_rec_set = [['ARCUS SPIRITS', 'SCHOOL RUMBLE', 'TRIPLE SCORE: 3 GAMES IN 1'], ['QUIZ TORIMONOCHOU', 'KEN TO MAHOU TO GAKUEN MONO. FINAL: SHINNYUSEI WA OHIMESAMA', 'ALESHAR: THE WORLD OF ICE'], ['WAVE BREAK', 'OLLIOLLI WORLD', 'SKATE 3']]

fake_game_rec_set = [["ALCHEMY GARDEN"], ["OVERCOOKED"], ['OLLIOLLI WORLD']]
for g in range(len(fake_game_rec_set)):
    t = []
    e = []
    for game in fake_game_rec_set[g]:
        t += GRMOD.getGameTags(game)
        e += GRMOD.getGameEntities(game)
    fake_games[g]['tags'] = t
    fake_games[g]['entities'] = e


#### MAKE NEW FEATURES

In [115]:
# get the transformer features for all of the games
for game in real_games:
    print(f"Getting features for {game['name']}")
    p = 0.8
    k = 100
    t = 0.99
    arg_set = {'top_k':k,'top_p':p, 'temperature':t}
    game['trans_features'] = predictTrans2(game['tags'], game['entities'], arg_set).split("\n")

for game in fake_games:
    print(f"Getting features for {game['name']}")
    p = 0.8
    k = 100
    t = 0.99
    arg_set = {'top_k':k,'top_p':p, 'temperature':t}
    ptok = getEntTags(game['prompt'])
    game['trans_features'] = predictTrans2(game['tags']+ptok[0], game['entities']+ptok[1], arg_set).split("\n")

Getting features for TEAM FORTRESS 2
k 100, p: 0.8, temp: 0.99
Getting features for DOTA 2
k 100, p: 0.8, temp: 0.99
Getting features for THE LEGEND OF ZELDA: BREATH OF THE WILD
k 100, p: 0.8, temp: 0.99
Getting features for ALCHEMY PRINCESS
k 100, p: 0.8, temp: 0.99
Getting features for THE ONIGIRI DORM
k 100, p: 0.8, temp: 0.99
Getting features for HOVERDECK 209X
k 100, p: 0.8, temp: 0.99


In [116]:
# get the feature generator features for all of the games
for game in tqdm(real_games):
    cnf, cnm = predictConNet2(game['entities'])
    game['cnet'] = cnm
    game['cnet_features'] = cnf

for game in tqdm(fake_games):
    tokens = utils.tokenize(game['prompt'])
    cnf, cnm = predictConNet2(game['entities']+tokens)
    game['cnet'] = cnm
    game['cnet_features'] = cnf

100%|██████████| 3/3 [00:36<00:00, 12.27s/it]


#### Print the results

In [117]:
def fprint(txt,reset=False):
    print(txt)
    if reset:
        with open("../data/feature_comp.txt", "w") as f:
            f.write(txt + "\n")
    else:
        with open("../data/feature_comp.txt", "a") as f:
            f.write(txt + "\n")

fprint("====== FEATURE COMPARISON ======\n\n",reset=True)

nl = "\n"
for game in real_games:
    fprint(f"Game: {game['name']}")
    fprint(f"Prompt: {game['prompt']}")

    hf = "\n".join(game['human_features'])
    tf = "\n".join(game['trans_features'])
    cf = "\n".join(game['cnet_features'])

    fprint(f"-- Human Features -- {nl}{hf}{nl}")
    fprint(f"-- Transformer Features -- {nl}{tf}{nl}")
    fprint(f"-- ConceptNet Features -- {nl}{cf}{nl}")

for game in fake_games:
    fprint(f"Game: {game['name']}")
    fprint(f"Prompt: {game['prompt']}")

    hf = "\n".join(game['human_features'])
    tf = "\n".join(game['trans_features'])
    cf = "\n".join(game['cnet_features'])

    fprint(f"-- Human Features -- {nl}{hf}{nl}")
    fprint(f"-- Transformer Features -- {nl}{tf}{nl}")
    fprint(f"-- ConceptNet Features -- {nl}{cf}{nl}")

====== FEATURE COMPARISON ======


Game: TEAM FORTRESS 2
Prompt: a class based multiplayer online shooter with a variety of modes including capture the flag, death match, and deliver the payload
-- Human Features -- 
- shoot gun to heal allies
- double jump in the air
- drink energy drinks
- throw suspicious jars
- backstab enemy targets
- sneak across terrain

-- Transformer Features -- 
action,free to play,multiplayer,first-person,funny,shooter,comedy,co-op,fps,cartoony,online co-op,tactical,crafting,cartoon,robots,team-based,competitive,class-based,hero shooter,trading,action,shooter
-,abilities,achievements,activity,assault,attempt,background,base,beam,bercharge,bomb,bunch,cart,cell,chain,checkpoints,class,classes,classics,commentary,control,defense,developer,elements,end,enemy,example,feature,game,gameplay,games,gun,healing,instant,kart,kills,loner,look,map,maps,medic,melee,menu,mini,mode,modes,multiplayer,offensive,order,overview,past,payload,player,points,rail,release,remake,rou

#### EXTRAS

In [130]:
for game in fake_games:
    print(f"Getting features for {game['name']}")
    p = 0.8
    k = 100
    t = 0.99
    arg_set = {'top_k':k,'top_p':p, 'temperature':t}
    print(predictTrans2(game['tags'], game['entities'], arg_set).split("\n"))

Getting features for ALCHEMY PRINCESS
k 100, p: 0.8, temp: 0.99
['atmospheric,fantasy,colorful,early access,cute,relaxing,family friendly,open world,sandbox,crafting,nature,life sim,farming sim,capitalism,agriculture,cozy,wholesome,adventure', 'alchemy,areas,bags,biomes,combinations,components,create,creations,experience,experiments,exploration,flowers,furniture,game,garden,gardening,gardens,gold,home,imagination,ingredients,kinds,land,mechanics,objects,piece,plants,potions,recipes,sandbox,seed,shop,storemanage,tools,trees,variety,villagers,world', '- unleash your imagination', '- explore, fill in hidden areas', '- discover new plants', '- fill up your shop', '- create unique gardens', '- explore the vast world', '- use plants', '- combine ingredients', '- Grow plants', '- explore a vast world', '- explore new ingredients', '- Use plants', '- create your own unique garden', '- use to create many kinds', '- create unique gardens', '- use plants', '- use plants', '- explore the world', '

In [148]:
fake_games[2]['cnet'].generate(10)

['solve problems',
 'meet over lunch',
 'needle you',
 'meet for coffee',
 'make you happy',
 'beat the dealer',
 'play a game of chess',
 'keep vehicles on the street',
 'build something wooden with',
 'jump on']

In [149]:
len(GRMOD.GAME_DATA.keys())

59771